![](https://github.com/SauravMaheshkar/Tabular-Playground-Series-May-2021/blob/main/assets/Banner.png?raw=true)

# Table of Content

1. [Packages 📦 and Basic Setup](#basic)
2. [Pre-Processing 👎🏻 -> 👍](#preprocess)
3. [The Model 👷‍♀️](#model)
4. [Training 💪🏻](#train)

## Disclaimer

This kernel builds on top of [@subinium](https://www.kaggle.com/subinium)'s kernel [TPS-May:Deeplearning Pipeline for Beginner](https://www.kaggle.com/subinium/tps-may-deeplearning-pipeline-for-beginner)

<a id = "basic"> </a>

# Packages 📦 and Basic Setup

In [ ]:
%%capture
!pip install wandb

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Weights and Biases
import wandb
from wandb.keras import WandbCallback
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=api_key);
wandb.init(project='TPS May 2021', entity='sauravmaheshkar')

As we can see, the dataset consists of 50 feature columnns with 4 classes. 

In [ ]:
# Basic Paths
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
train.head()

<a id = 'preprocess'> </a>
# Pre-Processing 👎🏻 -> 👍

In this kernel I'll highlight the relatively new `tensorflow.feature_column` submodule which contains a ton of useful methods to deal with structured data. For more details kindly visit the [documentation](https://www.tensorflow.org/api_docs/python/tf/feature_column). Tensorflow offers a ton of feature columns for us to experiment with, viz :

* Numeric columns
* Bucketized columns
* Categorical columns
* Embedding columns
* Hashed feature columns
* Crossed feature columns

Have a look at [this tutorial](https://www.tensorflow.org/tutorials/structured_data/feature_columns) for more details.

Upon a closer look, we realize that most features are left skewed in this dataset. Thus, Normalization seems ideal.

In [ ]:
for i in range(50):
    mean, std = train[f'feature_{i}'].mean(), train[f'feature_{i}'].std()
    train[f'feature_{i}'] = train[f'feature_{i}'].apply(lambda x : (x-mean)/std)

We also convert the `target` columns into binary class matrices using the `tf.keras.utils.to_categorical()` function.

In [ ]:
# transform target column into 0,1,2,3 values
label_dict = {val:idx for idx, val in enumerate(sorted(train['target'].unique()))}
train['target'] = train['target'].map(label_dict)

train['target'] = tf.keras.utils.to_categorical(train['target'])

Lastly, we split out dataset into train, validation and test splits. Most deep learning models will overfit, so having a nice split ratio is key 🔑

In [ ]:
train, test = train_test_split(train, test_size=0.2)
train, val = train_test_split(train, test_size=0.4)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

Rather than using dataframes we'll create a `tf.data.Dataset` from our original dataframe. This also allows us to efficiently shuffle, prefetch and create batches.

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

A numeric column is the simplest type of feature column. It is used to represent real valued features. When using this column, our model will receive the column value from the dataframe **unchanged**. The output of a feature column will become the input to the model.

In [ ]:
from tensorflow import feature_column

feature_columns = []

for i in range(50):
    feature_columns.append(feature_column.numeric_column(f'feature_{i}'))

<a id='model'></a>
# The Model 👷‍♀️

We create a `feature_layer`, which will act as the first layer in our model. A batch size of 64 was arbitrarily chosen.

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

We just add a Dense layer in the end to act as a classification head for the model and compile using the `sgd` optimizer and the `sparse_categorical_crossentropy` loss function.

In [ ]:
model = tf.keras.models.Sequential([
        feature_layer,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

<a id='train'></a>
# Training 💪🏻

We train for a mere 10 epochs and the overfitting is quite obvious here. Better Regularization strategies + better feature selection can be extremely helpful moving forward. We'll also log our metrics to [Weights and Biases](https://wandb.ai/site) for efficient experiment tracking.

In [ ]:
model.fit(train_ds,epochs = 10, verbose = 2,
          validation_data=val_ds,
          validation_steps = 100,
          callbacks = [WandbCallback()])

We can evaluate on our test set and see the loss and accuracy. As, we can see the model overfits to a large extent.

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)
print("Loss", loss)

# Submission

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
train_df = train_df.drop('id', axis = 1)
test_df = test_df.drop('id', axis=1)

for i in range(50):
    mean, std = train_df[f'feature_{i}'].mean(), train_df[f'feature_{i}'].std()
    test_df[f'feature_{i}'] = test_df[f'feature_{i}'].apply(lambda x : (x-mean)/std)

In [ ]:
def df_to_dataset_test(dataframe, batch_size=32):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

test_df = df_to_dataset_test(test_df, batch_size=batch_size)

In [ ]:
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = model.predict(test_df)
sample_submission.to_csv('densefeatures_submission.csv',index = False)
sample_submission.head()